In [1]:
import requests
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
import json
import calendar
import urllib

# Import the API key.
from config import Token_NOAA
from config import API_Token

#Suppress Warnings
pd.options.mode.chained_assignment = None  # default='warn'
#Display all columns
pd.set_option('display.max_columns', None)

In [2]:
# https://github.com/crvaden/NOAA_API_v2
# https://towardsdatascience.com/getting-weather-data-in-3-easy-steps-8dc10cc5c859
# https://cran.r-project.org/web/packages/rnoaa/rnoaa.pdf
# file:///C:/Users/15124/Downloads/GHCND_documentation.pdf
# https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt

In [3]:
# This JN finds the weather by the zip code.
# The API can be probamatic to say the least.
# It takes a while for this to run and a appellation can throw an error once and then work the next time.

# Temperature Data By Zip Code

In [4]:
temp = pd.DataFrame()
states = {
    'Napa': {
        'zip': '95472',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Walla': {
        'zip': '99362',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Columbia': {
        'zip': '98813',
        'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Sonoma': {
        'zip': '95476',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Santa': {
        'zip': '95062',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Yakima': {
        'zip': '98903',
        'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Dundee': {
        'zip': '97045',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Willamette': {
        'zip': '97302',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        }
}

In [5]:
for state in states:
    small_df = pd.DataFrame()
    for year in states[state]['years']:
        zip = states[state]['zip']
        df_test = pd.DataFrame()
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip}&datatypeid=TMAX&datatypeid=TMIN&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
        #Load JSON data
        d = json.loads(r.text)
        items_MAX = [item for item in d['results'] if item['datatype']=='TMAX']
        items_MIN = [item for item in d['results'] if item['datatype']=='TMIN']
        # #get the date field from all average temperature readings
        dates_temp_MAX = [item['date'].split('T')[0] for item in items_MAX]
        dates_temp_MIN = [item['date'].split('T')[0] for item in items_MIN]
        df_test['date'] = dates_temp_MAX
        df_test['avgMaxTemp'+state] = np.nan
        for item in items_MAX:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMaxTemp'+state]] = item['value']
        df_test['avgMinTemp'+state] = np.nan
        for item in items_MIN:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMinTemp'+state]] = item['value']
        small_df = pd.concat([small_df,df_test])
    if temp.empty:
        temp = small_df
    else:
        temp = pd.merge(temp,small_df)
temp.to_csv( 'RawDailyTemp.csv', index = False)

#### Resample Daily Summaries to Monthly Summaries by Resetting Index, and Converting Date to DateTime

In [6]:
temp=temp.set_index('date')
temp.reset_index(inplace=True)
temp['date']=pd.to_datetime(temp['date'])
temp = temp.set_index('date').resample('M').mean()
temp.head()

,avgMaxTempNapa,avgMinTempNapa,avgMaxTempWalla,avgMinTempWalla,avgMaxTempColumbia,avgMinTempColumbia,avgMaxTempSonoma,avgMinTempSonoma,avgMaxTempSanta,avgMinTempSanta,avgMaxTempYakima,avgMinTempYakima,avgMaxTempDundee,avgMinTempDundee,avgMaxTempWillamette,avgMinTempWillamette
date,,,,,,,,,,,,,,,,
1992-01-31,54.758621,34.586207,46.931034,35.103448,39.482759,26.379310,54.793103,35.034483,62.379310,38.655172,42.482759,27.448276,52.344828,39.793103,51.482759,37.689655
1992-02-29,63.964286,41.392857,49.928571,36.250000,46.357143,33.392857,64.250000,43.142857,65.214286,45.821429,49.464286,30.714286,58.428571,41.185185,56.607143,38.464286
1992-03-31,64.677419,43.967742,60.838710,39.935484,62.870968,36.322581,65.741935,44.032258,64.612903,47.838710,62.806452,33.387097,65.483871,42.290323,63.225806,37.806452
1992-04-30,72.933333,43.800000,65.733333,44.666667,67.166667,40.166667,74.433333,47.500000,72.800000,49.400000,65.100000,36.333333,65.066667,47.200000,64.300000,43.300000
1992-05-31,78.466667,47.166667,78.766667,48.933333,79.666667,48.000000,82.900000,49.800000,70.900000,52.400000,78.466667,42.133333,77.900000,49.833333,76.966667,44.300000


#### Convert Date from YYYY-MM-DD to Year and Month Column

In [7]:
temp.reset_index(inplace= True )
temp['date']=pd.to_datetime(temp['date'])
temp.reset_index(inplace=True)
temp['day'] = temp['date'].dt.day
temp['month'] = temp['date'].dt.month
temp['year'] = temp['date'].dt.year

list_months=[]
for x in temp['month']:
    list_months.append(calendar.month_name[x])
temp['months']=list_months
temp.drop(['date', 'day','month'], axis=1, inplace=True)
temp.head()

,index,avgMaxTempNapa,avgMinTempNapa,avgMaxTempWalla,avgMinTempWalla,avgMaxTempColumbia,avgMinTempColumbia,avgMaxTempSonoma,avgMinTempSonoma,avgMaxTempSanta,avgMinTempSanta,avgMaxTempYakima,avgMinTempYakima,avgMaxTempDundee,avgMinTempDundee,avgMaxTempWillamette,avgMinTempWillamette,year,months
0,0,54.758621,34.586207,46.931034,35.103448,39.482759,26.379310,54.793103,35.034483,62.379310,38.655172,42.482759,27.448276,52.344828,39.793103,51.482759,37.689655,1992,January
1,1,63.964286,41.392857,49.928571,36.250000,46.357143,33.392857,64.250000,43.142857,65.214286,45.821429,49.464286,30.714286,58.428571,41.185185,56.607143,38.464286,1992,February
2,2,64.677419,43.967742,60.838710,39.935484,62.870968,36.322581,65.741935,44.032258,64.612903,47.838710,62.806452,33.387097,65.483871,42.290323,63.225806,37.806452,1992,March
3,3,72.933333,43.800000,65.733333,44.666667,67.166667,40.166667,74.433333,47.500000,72.800000,49.400000,65.100000,36.333333,65.066667,47.200000,64.300000,43.300000,1992,April
4,4,78.466667,47.166667,78.766667,48.933333,79.666667,48.000000,82.900000,49.800000,70.900000,52.400000,78.466667,42.133333,77.900000,49.833333,76.966667,44.300000,1992,May


#### Average Max and Min Temperature

In [8]:
Final_napa_min_max=pd.DataFrame()
Final_napa_min_max['avgTempNapa']=(temp['avgMaxTempNapa']+temp['avgMinTempNapa'])/2
Final_napa_min_max['avgTempColumbia']=(temp['avgMaxTempColumbia']+temp['avgMinTempColumbia'])/2
Final_napa_min_max['avgTempWalla']=(temp['avgMaxTempWalla']+temp['avgMaxTempWalla'])/2
Final_napa_min_max['avgTempSonoma']=(temp['avgMaxTempSonoma']+temp['avgMaxTempSonoma'])/2
Final_napa_min_max['avgTempSanta']=(temp['avgMaxTempSanta']+temp['avgMinTempSanta'])/2
Final_napa_min_max['avgTempYakima']=(temp['avgMaxTempYakima']+temp['avgMinTempYakima'])/2
Final_napa_min_max['avgTempDundee']=(temp['avgMaxTempDundee']+temp['avgMinTempDundee'])/2
Final_napa_min_max['Year']=temp['year']
Final_napa_min_max['Months']=temp['months']
Final_napa_min_max

,avgTempNapa,avgTempColumbia,avgTempWalla,avgTempSonoma,avgTempSanta,avgTempYakima,avgTempDundee,Year,Months
0,44.672414,32.931034,46.931034,54.793103,50.517241,34.965517,46.068966,1992,January
1,52.678571,39.875000,49.928571,64.250000,55.517857,40.089286,49.806878,1992,February
2,54.322581,49.596774,60.838710,65.741935,56.225806,48.096774,53.887097,1992,March
3,58.366667,53.666667,65.733333,74.433333,61.100000,50.716667,56.133333,1992,April
4,62.816667,63.833333,78.766667,82.900000,61.650000,60.300000,63.866667,1992,May
...,...,...,...,...,...,...,...,...,...
303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,April
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,May
305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,June
306,67.666667,78.000000,96.333333,85.333333,64.000000,80.500000,73.333333,2017,July


#### Count NA Values

In [9]:
print(Final_napa_min_max.isnull().sum())

avgTempNapa        82
avgTempColumbia    82
avgTempWalla       82
avgTempSonoma      82
avgTempSanta       82
avgTempYakima      82
avgTempDundee      82
Year                0
Months              0
dtype: int64


# Precipitation Data By Zip Code

In [10]:
prcp = pd.DataFrame()
states = {
    'Napa': {
        'zip': '94559',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Walla': {
        'zip': '99362',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Columbia': {
        'zip': '98813',
        'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Sonoma': {
        'zip': '95476',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Santa': {
        'zip': '95062',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Yakima': {
        'zip': '98903',
        'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Dundee': {
        'zip': '97148',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Willamette': {
        'zip': '97302',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        }
}

In [11]:
for state in states:
    small_df = pd.DataFrame()
    for year in states[state]['years']:
        zip = states[state]['zip']
        df_test = pd.DataFrame()
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip}&datatypeid=TMAX&datatypeid=PRCP&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
        #Load JSON data
        d = json.loads(r.text)
#         print(d)
        items_MAX = [item for item in d['results'] if item['datatype']=='PRCP']
        # #get the date field from all average temperature readings
        dates_temp_MAX = [item['date'].split('T')[0] for item in items_MAX]
        df_test['date'] = dates_temp_MAX
        df_test['PRCP'+state] = np.nan
        for item in items_MAX:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['PRCP'+state]] = item['value']
        small_df = pd.concat([small_df,df_test])
    if prcp.empty:
        prcp = small_df
    else:
        prcp = pd.merge(prcp,small_df)
#     print(list(napadf.columns))
prcp.to_csv( 'RawDailyPrcp.csv', index = False)

#### Resample Daily Summaries to Monthly Summaries by Resetting Index, and Converting Date to DateTime¶

In [12]:
prcp=prcp.set_index('date')
prcp.reset_index(inplace=True)
prcp['date']=pd.to_datetime(prcp['date'])
prcp = prcp.set_index('date').resample('M').mean()

#### Convert Date from YYYY-MM-DD to Year and Month Column

In [13]:
prcp.reset_index(inplace= True )
prcp['date']=pd.to_datetime(prcp['date'])
prcp.reset_index(inplace=True)
prcp['day'] = prcp['date'].dt.day
prcp['month'] = prcp['date'].dt.month
prcp['year'] = prcp['date'].dt.year

list_months=[]
for x in prcp['month']:
    list_months.append(calendar.month_name[x])
prcp['months']=list_months
prcp.drop(['date', 'day','month'], axis=1, inplace=True)
prcp

,index,PRCPNapa,PRCPWalla,PRCPColumbia,PRCPSonoma,PRCPSanta,PRCPYakima,PRCPDundee,PRCPWillamette,year,months
0,0,0.068861,0.050506,0.024430,0.068734,0.093797,0.008734,0.403291,0.160886,1992,January
1,1,0.263590,0.047564,0.014103,0.347179,0.407821,0.022436,0.247051,0.151026,1992,February
2,2,0.138065,0.015806,0.005161,0.226129,0.140323,0.013226,0.043548,0.028710,1992,March
3,3,0.021000,0.067667,0.028000,0.030000,0.009333,0.032000,0.266667,0.145000,1992,April
4,4,0.000000,0.008387,0.000323,0.000000,0.000000,0.001290,0.001290,0.001613,1992,May
...,...,...,...,...,...,...,...,...,...,...,...
298,298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,November
299,299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,December
300,300,0.497898,0.046426,0.055149,0.405962,0.569689,0.070461,0.481832,0.188265,2017,January
301,301,0.457407,0.092029,0.083613,0.353525,0.518903,0.086728,0.621005,0.473552,2017,February


#### Count NA Values

In [14]:
print(prcp.isnull().sum())

index               0
PRCPNapa          130
PRCPWalla         130
PRCPColumbia      130
PRCPSonoma        130
PRCPSanta         130
PRCPYakima        130
PRCPDundee        130
PRCPWillamette    130
year                0
months              0
dtype: int64


# Create Weather CSV Files

In [15]:
Final_napa_min_max.to_csv( 'RawTemp_Weather.csv', index = False)
prcp.to_csv( 'RawPrcp_Weather.csv', index = False)

# Import Red and White Wine

In [16]:
# Import the API key.
headers = {
    'accept': 'application/json',
    'authorization': API_Token
    }

## Red Wine

In [17]:
#import JSON data with the API
req_red_USA = urllib.request.Request('https://api.globalwinescore.com/globalwinescores/latest/?color=red&limit=5000&country=Usa', data=None, headers=headers, origin_req_host=None, unverifiable=False, method=None)
res_red_USA = urllib.request.urlopen(req_red_USA).read().decode()
obj_red_USA = json.loads(res_red_USA)
#read json into DF
red_USA_df = pd.DataFrame.from_dict(obj_red_USA['results'], orient='columns')
# Delete unnecessary columns
red_USA_df.drop([ 'wine_slug', 'appellation_slug',  'date',  'lwin', 'lwin_11'],axis=1, inplace=True)
#convert vintage to date type
red_USA_df['vintage']= pd.to_datetime(red_USA_df['vintage']).dt.year
red_USA_df['regions']=red_USA_df['regions'].str.get(0)
red_USA_df.to_csv("RawRed_Wine_USA.csv")
red_USA_df.head()

,wine,wine_id,appellation,color,wine_type,regions,country,classification,vintage,is_primeurs,score,confidence_index,journalist_count
0,"Chateau Montelena, Cabernet Sauvignon, Napa Va...",39440,Napa Valley,Red,,California,Usa,None,1991,False,94.78,C+,4
1,"Ridge Vineyards, Monte Bello, Santa Cruz Mount...",120786,Santa Cruz Mountains,Red,,California,Usa,None,2013,False,95.83,C+,8
2,"Quintessa, Rutherford",118360,Rutherford,Red,,California,Usa,None,2015,False,95.77,C,3
3,"Harlan Estate, Napa Valley",81980,Napa Valley,Red,,California,Usa,None,2001,False,98.89,A+,6
4,"Dalla Valle Vineyards, Maya, Napa Valley",48290,Napa Valley,Red,,California,Usa,None,2001,False,96.49,B+,5


## White WIne

In [18]:
#import JSON data with the API
req_white_USA = urllib.request.Request('https://api.globalwinescore.com/globalwinescores/latest/?color=white&limit=5000&country=Usa', data=None, headers=headers, origin_req_host=None, unverifiable=False, method=None)
res_white_USA = urllib.request.urlopen(req_white_USA).read().decode()
obj_white_USA = json.loads(res_white_USA)
#read json into DF
white_USA_df = pd.DataFrame.from_dict(obj_white_USA['results'], orient='columns')
# Delete unnecessary columns
white_USA_df.drop([ 'wine_slug', 'appellation_slug',  'date',  'lwin', 'lwin_11'],axis=1, inplace=True)
#convert vintage to date type
white_USA_df['vintage']= pd.to_datetime(white_USA_df['vintage']).dt.year
white_USA_df['regions']=white_USA_df['regions'].str.get(0)
white_USA_df.to_csv("RawWhite_Wine_USA.csv")
white_USA_df.head()

,wine,wine_id,appellation,color,wine_type,regions,country,classification,vintage,is_primeurs,score,confidence_index,journalist_count
0,"Marcassin Estate, Chardonnay, White, Sonoma Coast",101634,Sonoma Coast,White,dry,California,Usa,None,2009,False,93.08,C+,3
1,"Ridge Vineyards, Monte Bello Chardonnay, White...",120785,Santa Cruz Mountains,White,dry,California,Usa,None,2016,False,94.39,B,3
2,"Stags' Leap Winery, Chardonnay, White, Napa Va...",129823,Napa Valley,White,dry,California,Usa,None,2016,False,89.83,A+,3
3,"Joseph Phelps Vineyards, Freestone Chardonnay,...",89569,Sonoma Coast,White,dry,California,Usa,None,2016,False,90.73,C+,4
4,"Crossbarn By Paul Hobbs, Chardonnay, White, So...",47377,Sonoma Coast,White,dry,California,Usa,None,2016,False,90.39,B+,3


In [19]:
red_USA_df.to_csv( 'RawRedWine.csv', index = False)
white_USA_df.to_csv( 'RawWhiteWine.csv', index = False)